Initialize notebook for Spark

In [1]:
import findspark
findspark.init()

## Import required librries

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

Initialize SparkContext and SparkSession

In [3]:
sc=SparkContext.getOrCreate()
ss=SparkSession(sc)

## Load Sample Data

In [4]:
data=ss.read.csv("../data/titanic.csv", inferSchema=True, header=True)

Show Data Structure

In [5]:
data.printSchema()

root
 |-- pclass: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- cabin: string (nullable = true)
 |-- embarked: string (nullable = true)
 |-- boat: string (nullable = true)
 |-- body: integer (nullable = true)
 |-- home.dest: string (nullable = true)



Show Data Types

In [6]:
data.dtypes

[('pclass', 'int'),
 ('survived', 'int'),
 ('name', 'string'),
 ('sex', 'string'),
 ('age', 'double'),
 ('sibsp', 'int'),
 ('parch', 'int'),
 ('ticket', 'string'),
 ('fare', 'double'),
 ('cabin', 'string'),
 ('embarked', 'string'),
 ('boat', 'string'),
 ('body', 'int'),
 ('home.dest', 'string')]

Register a DataFrame as an SQL Temporary View

In [7]:
data.createOrReplaceTempView('titanic')

Select 5 records

In [8]:
ss.sql("SELECT * FROM titanic limit 5").show()

+------+--------+--------------------+------+------+-----+-----+------+--------+-------+--------+----+----+--------------------+
|pclass|survived|                name|   sex|   age|sibsp|parch|ticket|    fare|  cabin|embarked|boat|body|           home.dest|
+------+--------+--------------------+------+------+-----+-----+------+--------+-------+--------+----+----+--------------------+
|     1|       1|Allen, Miss. Elis...|female|  29.0|    0|    0| 24160|211.3375|     B5|       S|   2|null|        St Louis, MO|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|113781|  151.55|C22 C26|       S|  11|null|Montreal, PQ / Ch...|
|     1|       0|Allison, Miss. He...|female|   2.0|    1|    2|113781|  151.55|C22 C26|       S|null|null|Montreal, PQ / Ch...|
|     1|       0|Allison, Mr. Huds...|  male|  30.0|    1|    2|113781|  151.55|C22 C26|       S|null| 135|Montreal, PQ / Ch...|
|     1|       0|Allison, Mrs. Hud...|female|  25.0|    1|    2|113781|  151.55|C22 C26|       S|

Get number of Passengers in Each Class

In [9]:
ss.sql("SELECT pclass AS Passenger_Class, COUNT(*) AS Number_of_Passengers FROM titanic GROUP BY pclass").show()

+---------------+--------------------+
|Passenger_Class|Number_of_Passengers|
+---------------+--------------------+
|           null|                   1|
|              1|                 323|
|              3|                 709|
|              2|                 277|
+---------------+--------------------+



## Mathematical Functions

Get First, Last, Min, Max, Mean and Total fare of Passenger

In [10]:
ss.sql("SELECT FIRST(fare) AS First_Record, LAST(fare) AS Last_Record,MIN(fare) AS Min_Fare, MAX(fare) AS Max_Fare,AVG(fare) AS Average_fare, SUM(fare) AS Total_fare FROM titanic").show()

+------------+-----------+--------+--------+-----------------+----------------+
|First_Record|Last_Record|Min_Fare|Max_Fare|     Average_fare|      Total_fare|
+------------+-----------+--------+--------+-----------------+----------------+
|    211.3375|       null|     0.0|512.3292|33.29547928134572|43550.4869000002|
+------------+-----------+--------+--------+-----------------+----------------+



Get Standard Deviation of the fare for passengers

In [11]:
ss.sql("SELECT stddev(fare) AS Fare_Standard_Deviation FROM titanic").show()

+-----------------------+
|Fare_Standard_Deviation|
+-----------------------+
|      51.75866823917421|
+-----------------------+



Get Variance of fare distribution of the passengers

In [12]:
ss.sql("SELECT variance(fare) AS Fare_Variance FROM titanic").show()

+------------------+
|     Fare_Variance|
+------------------+
|2678.9597378929016|
+------------------+



Get Skewness of fare distribution of the passenger

In [13]:
ss.sql("SELECT skewness(fare) FROM titanic").show()

+-----------------+
|   skewness(fare)|
+-----------------+
|4.362698696939598|
+-----------------+



Get kurtosis of the fare distribution

In [14]:
ss.sql("SELECT kurtosis(fare) AS Kurtosis FROM titanic").show()

+-----------------+
|         Kurtosis|
+-----------------+
|26.92019523184775|
+-----------------+



## Analytical Functions

In [15]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [16]:
df=data['pclass','name','survived','fare']

row number Window Function. This function returns sequential row number starting from 1 to the result of each window partition

In [17]:
win_function=Window.partitionBy("pclass").orderBy("fare")

In [18]:
df.withColumn("Row_Number",row_number().over(win_function)).show(truncate=False)

+------+----------------------------------------------------+--------+-------+----------+
|pclass|name                                                |survived|fare   |Row_Number|
+------+----------------------------------------------------+--------+-------+----------+
|null  |null                                                |null    |null   |1         |
|1     |Andrews, Mr. Thomas Jr                              |0       |0.0    |1         |
|1     |Chisholm, Mr. Roderick Robert Crispin               |0       |0.0    |2         |
|1     |Fry, Mr. Richard                                    |0       |0.0    |3         |
|1     |Harrison, Mr. William                               |0       |0.0    |4         |
|1     |Ismay, Mr. Joseph Bruce                             |1       |0.0    |5         |
|1     |Parr, Mr. William Henry Marsh                       |0       |0.0    |6         |
|1     |Reuchlin, Jonkheer. John George                     |0       |0.0    |7         |
|1     |Ca

rank() window function. This function returns a rank to the result within a window partition while leaving gaps where there is duplicates

In [19]:
from pyspark.sql.functions import rank

df.withColumn("Rank", rank().over(win_function)).show(truncate=False)

+------+----------------------------------------------------+--------+-------+----+
|pclass|name                                                |survived|fare   |Rank|
+------+----------------------------------------------------+--------+-------+----+
|null  |null                                                |null    |null   |1   |
|1     |Andrews, Mr. Thomas Jr                              |0       |0.0    |1   |
|1     |Chisholm, Mr. Roderick Robert Crispin               |0       |0.0    |1   |
|1     |Fry, Mr. Richard                                    |0       |0.0    |1   |
|1     |Harrison, Mr. William                               |0       |0.0    |1   |
|1     |Ismay, Mr. Joseph Bruce                             |1       |0.0    |1   |
|1     |Parr, Mr. William Henry Marsh                       |0       |0.0    |1   |
|1     |Reuchlin, Jonkheer. John George                     |0       |0.0    |1   |
|1     |Carlsson, Mr. Frans Olof                            |0       |5.0   

dense_rank() window function. This function returns a rank to the result within a window partition without leaving gaps where there is duplicates

In [20]:
from pyspark.sql.functions import dense_rank

df.withColumn("Dense_Rank",dense_rank().over(win_function)).show(truncate=False)

+------+----------------------------------------------------+--------+-------+----------+
|pclass|name                                                |survived|fare   |Dense_Rank|
+------+----------------------------------------------------+--------+-------+----------+
|null  |null                                                |null    |null   |1         |
|1     |Andrews, Mr. Thomas Jr                              |0       |0.0    |1         |
|1     |Chisholm, Mr. Roderick Robert Crispin               |0       |0.0    |1         |
|1     |Fry, Mr. Richard                                    |0       |0.0    |1         |
|1     |Harrison, Mr. William                               |0       |0.0    |1         |
|1     |Ismay, Mr. Joseph Bruce                             |1       |0.0    |1         |
|1     |Parr, Mr. William Henry Marsh                       |0       |0.0    |1         |
|1     |Reuchlin, Jonkheer. John George                     |0       |0.0    |1         |
|1     |Ca

percent_rank Window Function. The PERCENT_RANK function computes the rank of the passenger's fare within a passenger class (pclass) as a percentage

In [21]:
from pyspark.sql.functions import percent_rank
df.withColumn("Percent_Rank",percent_rank().over(win_function)).show(truncate=False)

+------+----------------------------------------------------+--------+-------+--------------------+
|pclass|name                                                |survived|fare   |Percent_Rank        |
+------+----------------------------------------------------+--------+-------+--------------------+
|null  |null                                                |null    |null   |0.0                 |
|1     |Andrews, Mr. Thomas Jr                              |0       |0.0    |0.0                 |
|1     |Chisholm, Mr. Roderick Robert Crispin               |0       |0.0    |0.0                 |
|1     |Fry, Mr. Richard                                    |0       |0.0    |0.0                 |
|1     |Harrison, Mr. William                               |0       |0.0    |0.0                 |
|1     |Ismay, Mr. Joseph Bruce                             |1       |0.0    |0.0                 |
|1     |Parr, Mr. William Henry Marsh                       |0       |0.0    |0.0                 |


cume_dist Window Function. This function returns the cumulative distribution of values within a window partition

In [22]:
from pyspark.sql.functions import cume_dist
df.withColumn("Cummulative_Dist",cume_dist().over(win_function)).show(truncate=False)

+------+----------------------------------------------------+--------+-------+--------------------+
|pclass|name                                                |survived|fare   |Cummulative_Dist    |
+------+----------------------------------------------------+--------+-------+--------------------+
|null  |null                                                |null    |null   |1.0                 |
|1     |Andrews, Mr. Thomas Jr                              |0       |0.0    |0.021671826625386997|
|1     |Chisholm, Mr. Roderick Robert Crispin               |0       |0.0    |0.021671826625386997|
|1     |Fry, Mr. Richard                                    |0       |0.0    |0.021671826625386997|
|1     |Harrison, Mr. William                               |0       |0.0    |0.021671826625386997|
|1     |Ismay, Mr. Joseph Bruce                             |1       |0.0    |0.021671826625386997|
|1     |Parr, Mr. William Henry Marsh                       |0       |0.0    |0.021671826625386997|


lag Window Function. Returns the previous value

In [23]:
from pyspark.sql.functions import lag

df.withColumn("Previous_Fare",lag("fare",1).over(win_function)).show()

+------+--------------------+--------+-------+-------------+
|pclass|                name|survived|   fare|Previous_Fare|
+------+--------------------+--------+-------+-------------+
|  null|                null|    null|   null|         null|
|     1|Andrews, Mr. Thom...|       0|    0.0|         null|
|     1|Chisholm, Mr. Rod...|       0|    0.0|          0.0|
|     1|    Fry, Mr. Richard|       0|    0.0|          0.0|
|     1|Harrison, Mr. Wil...|       0|    0.0|          0.0|
|     1|Ismay, Mr. Joseph...|       1|    0.0|          0.0|
|     1|Parr, Mr. William...|       0|    0.0|          0.0|
|     1|Reuchlin, Jonkhee...|       0|    0.0|          0.0|
|     1|Carlsson, Mr. Fra...|       0|    5.0|          0.0|
|     1|Colley, Mr. Edwar...|       0|25.5875|          5.0|
|     1|Cornell, Mrs. Rob...|       1|   25.7|      25.5875|
|     1|Omont, Mr. Alfred...|       1|25.7417|         25.7|
|     1| Baumann, Mr. John D|       0| 25.925|      25.7417|
|     1|Leader, Dr. Alic

lead Window Function. Returns the next value

In [24]:
from pyspark.sql.functions import lead

df.withColumn("Next_Fare",lead("fare",1).over(win_function)).show(truncate=False)

+------+----------------------------------------------------+--------+-------+---------+
|pclass|name                                                |survived|fare   |Next_Fare|
+------+----------------------------------------------------+--------+-------+---------+
|null  |null                                                |null    |null   |null     |
|1     |Andrews, Mr. Thomas Jr                              |0       |0.0    |0.0      |
|1     |Chisholm, Mr. Roderick Robert Crispin               |0       |0.0    |0.0      |
|1     |Fry, Mr. Richard                                    |0       |0.0    |0.0      |
|1     |Harrison, Mr. William                               |0       |0.0    |0.0      |
|1     |Ismay, Mr. Joseph Bruce                             |1       |0.0    |0.0      |
|1     |Parr, Mr. William Henry Marsh                       |0       |0.0    |0.0      |
|1     |Reuchlin, Jonkheer. John George                     |0       |0.0    |5.0      |
|1     |Carlsson, Mr.